[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sotirischatzimiltis/MscThesis/blob/main/SpikingNeuralNetwork/spiking_neural_network_muliclass_classification.ipynb)

# Dataset

In [1]:
train_Dataset = 'train_rf05_multiclass.csv'
test_Dataset = 'test_rf05_multiclass.csv'

# Set Training Arguments

In [2]:
bsize =1024 # set batch size 
learning_rate = 0.0005 # set learning rate of optimizer  
num_epochs = 2 # number of epochs
num_hidden = 200 # hidden layer neurons 

# Temporal Dynamics
num_steps = 25
beta = 0.9

In [ ]:
!pip install snntorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# imports
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import itertools
import pandas as pd 

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sn

# 1. Setting up the Dataset

Get data from GitLab

In [ ]:
class MyDataset(Dataset):
  def __init__(self,file_name):
    df=pd.read_csv(file_name)

    x=df.iloc[:,:-1].values
    y=df.iloc[:,-1].values
    print(x)
    print(y)
    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y_train=torch.tensor(y,dtype=torch.long)

  def __len__(self):
    return len(self.y_train)
  
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [ ]:
batch_size = bsize
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

trainDs=MyDataset(train_Dataset)
testDs=MyDataset(test_Dataset)

train_loader = DataLoader(trainDs, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(testDs, batch_size=batch_size, shuffle=True, drop_last=True)

# 2. Define the Network

In [ ]:
# Network Architecture
num_inputs = len(trainDs.x_train[1]) # input layer neurons i
num_outputs = 5 # output layer neurons  

In [ ]:
# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_hidden)
        self.lif2 = snn.Leaky(beta=beta)
        self.fc3 = nn.Linear(num_hidden,num_outputs)
        self.lif3 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        
        # Record the final layer
        spk3_rec = []
        mem3_rec = []

        for step in range(num_steps):
            cur1 = self.fc1(x)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            cur3 = self.fc3(spk2)
            spk3, mem3 = self.lif3(cur3,mem3)

            spk3_rec.append(spk3)
            mem3_rec.append(mem3)

        return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)
        
# Load the network onto CUDA if available
net = Net().to(device)

The code in the `forward()` function will only be called once the input argument `x` is explicitly passed into `net`.

* `fc1` applies a linear transformation to all input pixels from the MNIST dataset;
* `lif1` integrates the weighted input over time, emitting a spike if the threshold condition is met;
* `fc2` applies a linear transformation to the output spikes of `lif1`;
* `lif2` is another spiking neuron layer, integrating the weighted spikes over time.

# 3. Training the SNN

## 3.1 Accuracy Metric
Below is a function that takes a batch of data, counts up all the spikes from each neuron (i.e., a rate code over the simulation time), and compares the index of the highest count with the actual target. If they match, then the network correctly predicted the target.

In [ ]:
# pass data into the network, sum the spikes over time
# and compare the neuron with the highest number of spikes
# with the target

def print_batch_accuracy(data, targets, train=False):
    output, _ = net(data.view(batch_size, -1))
    ######check#########
    _, idx = output.sum(dim=0).max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

def train_printer():
    print(f"Epoch {epoch}, Iteration {iter_counter}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(data, targets, train=True)
    print_batch_accuracy(test_data, test_targets, train=False)
    print("\n")

## 3.2 Loss Definition
The `nn.CrossEntropyLoss` function in PyTorch automatically handles taking the softmax of the output layer as well as generating a loss at the output. 

In [ ]:
loss = nn.CrossEntropyLoss()

## 3.3 Optimizer
Adam is a robust optimizer that performs well on recurrent networks, so let's use that with a learning rate of $5\times10^{-4}$. 

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, betas=(0.9, 0.999))

## 3.4 Training Loop

Let's combine everything into a training loop. We will train for one epoch (though feel free to increase `num_epochs`), exposing our network to each sample of data once.

In [ ]:
loss_hist = []
test_loss_hist = []
counter = 0

import time
start_time = time.time()
trainloss = np.zeros(num_epochs)
testloss = np.zeros(num_epochs)
trainacc = np.zeros(num_epochs)
testacc = np.zeros(num_epochs)

# Outer training loop
for epoch in range(num_epochs):
    train_loss = 0.0
    teloss = 0.0
    train_acc =0.0
    test_acc = 0.0
    iter_counter = 0
    train_batch = iter(train_loader)
    train_samples = 0
    test_samples = 0
    # Minibatch training loop
    for data, targets in train_batch:
        train_samples += 1
        #spike_data = spikegen.rate(data, num_steps=num_steps)
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        net.train()
        spk_rec, mem_rec = net(data.view(batch_size, -1))
        _, idx = spk_rec.sum(dim=0).max(1)
        acc = np.mean((targets == idx).detach().cpu().numpy())
        train_acc+= acc
        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
            loss_val += loss(mem_rec[step], targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())
        train_loss += loss_val.item()
        # Test set
        with torch.no_grad():
            net.eval()
            test_data, test_targets = next(iter(test_loader))
            #spike_test_data = spikegen.rate(test_data,num_steps =num_steps)
            test_data = test_data.to(device)
            test_targets = test_targets.to(device)
            test_samples += 1

            # Test set forward pass
            test_spk, test_mem = net(test_data.view(batch_size, -1))
            _, idx = test_spk.sum(dim=0).max(1)
            tacc = np.mean((test_targets == idx).detach().cpu().numpy())
            test_acc += tacc
            # Test set loss
            test_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                test_loss += loss(test_mem[step], test_targets)
            test_loss_hist.append(test_loss.item())
            teloss += test_loss.item()
            # Print train/test loss/accuracy
            if counter % 50 == 0:
                train_printer()
            counter += 1
            iter_counter +=1
    trainloss[epoch] = train_loss 
    testloss[epoch] = teloss
    trainacc[epoch] = train_acc/train_samples
    testacc[epoch] = test_acc/ test_samples
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
for i in range(0,len(trainloss)):
  print(trainloss[i])

print("test--------------------------------------------------------------------")

for i in range(0,len(testloss)):
  print(testloss[i])

In [ ]:
fig = plt.figure()
plt.plot(trainloss)
plt.plot(testloss)
plt.title("Train & Test Loss Curves")
plt.legend(["Train Loss","Test Loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(trainacc)
plt.plot(testacc)
plt.title("Train & Test Accuracy Curves")
plt.legend(["Train Accuracy","Test Accuracy"])
plt.xlabel("Epoch")
plt.ylabel("Accurracy")
plt.show()

# 4. Results
## 4.1 Plot Training/Test Loss

In [ ]:
# Plot Loss
fig = plt.figure(facecolor="w", figsize=(10, 5))
plt.plot(loss_hist)
plt.plot(test_loss_hist)
plt.title("Loss Curves")
plt.legend(["Train Loss", "Test Loss"])
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

The loss curves are noisy because the losses are tracked at every iteration, rather than averaging across multiple iterations. 

## 4.2 Test Set Accuracy
This function iterates over all minibatches to obtain a measure of accuracy over the full samples in the test set.

In [ ]:
total = 0
correct = 0
y_pred = [] # for confusion matrix 
y_true = [] # for confusion matrix 
# drop_last switched to False to keep all samples
test_loader = DataLoader(testDs, batch_size=batch_size, shuffle=True, drop_last=False)

with torch.no_grad(): # switch off gradient computation
  net.eval() # switch for evaluation 
  for data, targets in test_loader: # load batch of data and targets
    data = data.to(device) # sent to gpu
    targets = targets.to(device) # sent to gpu 
    
    # forward pass
    test_spk, _ = net(data.view(data.size(0), -1)) # forward pass to predict

    # calculate total accuracy
    _, predicted = test_spk.sum(dim=0).max(1)
    y_pred.extend(predicted.cpu()) # Save Prediction
    y_true.extend(targets.cpu())
    total += targets.size(0)
    correct += (predicted == targets).sum().item()
    
print(f"Total correctly classified instances: {correct}/{total}")
print(f"Test Set Accuracy: {100 * correct / total:.2f}%")

# constant for classes
classes = ('Normal', 'DoS','Probe','R2L','U2R')

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)

# # rows is the truth , columns is the predicted 
# plot confusion matrix with actual numbers
df_cm= pd.DataFrame(cf_matrix, index = [i for i in classes],
                     columns = [i for i in classes])


plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True,fmt ='g')

print("Classification Report")
print(classification_report(y_true,y_pred,target_names = classes))


In [ ]:
total = 0
correct = 0
y_pred = [] # for confusion matrix 
y_true = [] # for confusion matrix 
# drop_last switched to False to keep all samples
train_loader = DataLoader(trainDs, batch_size=batch_size, shuffle=True, drop_last=False)

with torch.no_grad(): # switch off gradient computation
  net.eval() # switch for evaluation 
  for data, targets in train_loader: # load batch of data and targets
    data = data.to(device) # sent to gpu
    targets = targets.to(device) # sent to gpu 
    
    # forward pass
    train_spk, _ = net(data.view(data.size(0), -1)) # forward pass to predict 

    # calculate total accuracy
    _, predicted = train_spk.sum(dim=0).max(1)
    y_pred.extend(predicted.cpu()) # Save Prediction
    y_true.extend(targets.cpu())
    total += targets.size(0)
    correct += (predicted == targets).sum().item()

print(f"Total correctly classified instances: {correct}/{total}")
print(f"Test Set Accuracy: {100 * correct / total:.2f}%")

# constant for classes
classes = ('Normal', 'DoS','Probe','R2L','U2R')

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
# rows is the truth , columns is the predicted 
# plot confusion matrix with actual numbers
df_cm= pd.DataFrame(cf_matrix, index = [i for i in classes],
                     columns = [i for i in classes])

plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True,fmt ='g')

print("Classification Report")
print(classification_report(y_true,y_pred,target_names = classes))